# Capstone Project
Author: Izzie Nielsen

### Data Importing
Importing data from Human vs. AI Generated Essays dataset

##### Steps:


*   Upload the the dataset to Colab
*   Ensure the unzip file name matches your file name
*   Unzip the file
*   Read the csv
*   Store essays and labels in a dataframe



Note: will also allow user input eventually

In [9]:
!unzip archive.zip

Archive:  archive.zip
replace balanced_ai_human_prompts.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: balanced_ai_human_prompts.csv  


In [10]:
import pandas as pd

df = pd.read_csv('/content/balanced_ai_human_prompts.csv')

display(df.head())
display(df.columns)

texts = df['text']

y = df['generated']

,text,generated
0,"Machine learning, a subset of artificial intel...",1
1,"A decision tree, a prominent machine learning ...",1
2,"Education, a cornerstone of societal progress,...",1
3,"Computers, the backbone of modern technology, ...",1
4,"Chess, a timeless game of strategy and intelle...",1


Index(['text', 'generated'], dtype='object')

### Tokenization and Text Pre-Processing

Preparing the text before inputing into the models. First, the data is split into test and training (80/20 split). Then the training and testing esssays (X) are tokenized by the BertTokenizer. SVM and BERT have different vectorization requirements.

For the BERT model:
* Save the training and testing tokens into a dataset
* Add attention mask to help the model ignore unimportant tokens
* Add labels to the dataset for supervised learning

For SVM:
* Use TF-IDF vectorization to assign values to the words
* Remove stop words




In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42)


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
no_stops = [t for t in texts if t not in stopwords.words('english')]


## BERT Model Training

* Use transfer learning with pre-trained NLP model, BERT created by Google

In [12]:
#first tokenize, uses Bert Tokenizer
from transformers import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_tokens = bert_tokenizer(X_train.tolist(), truncation=True, padding='max_length', max_length=1024, return_tensors='pt')
test_tokens = bert_tokenizer(X_test.tolist(), truncation=True, padding='max_length', max_length=1024, return_tensors='pt')


KeyboardInterrupt: 

In [ ]:
#take tokens from BertTokenizer and convert to dataset that BERT can use
#adds attention mask - which tokens to ignore
#dataset from HuggingFace
from datasets import Dataset

train_dataset = Dataset.from_dict({
    'input_ids': train_tokens['input_ids'],
    'attention_mask': train_tokens['attention_mask'],
    'labels': y_train.tolist()
})

test_dataset = Dataset.from_dict({
    'input_ids': test_tokens['input_ids'],
    'attention_mask': test_tokens['attention_mask'],
    'labels': y_test.tolist()
})

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

## SVM

In [15]:
#takes tokens and assigns values
#only used by SVM
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=5000
)

# Fit the vectorizer on the training data and transform it
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the testing data using the fitted vectorizer
X_test_tfidf = vectorizer.transform(X_test)

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Initialize and train the SVM model with a linear kernel
model = SVC(kernel='linear', C=1000.0)  # High C value for hard margin
model.fit(X_train_tfidf, y_train)

SVC(C=1000.0, kernel='linear')

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Predict on the TF-IDF transformed testing data
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.9963636363636363
Precision: 1.0
Recall: 0.9924812030075187
